In [ ]:
import torch
import os
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc, confusion_matrix, roc_auc_score
from sklearn.metrics import confusion_matrix
import numpy as np
from tqdm import tqdm as tqdm
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.models as models

In [ ]:
# Define necessary paths
results_path = 'The path to the output text results file'

original_folders = ['set the path to the data files you want to compute the remaining utility'
                   ]

model_dir = 'The pathe to the Race classification model'

In [ ]:
dataset_dirs = original_folders

In [ ]:
# Define directories
classes = ['Asian_Chinese_Korean', 'Asian_Indian', 'Black', 'White']
num_classes = 4

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    ])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = models.convnext_base(pretrained=True)
model.classifier[2]=nn.Linear(1024,num_classes)

model.load_state_dict(torch.load(model_dir))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.00)

In [ ]:
def evaluate_model(model, test_loader, dataset_name):
    model.eval()  # Set the model to evaluation mode

    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, targets in tqdm(test_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            y_true.extend(targets.cpu().tolist())  # Extend the y_true list
            y_pred.extend(predicted.cpu().tolist())  # Extend the y_pred list

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')

    class_f1_scores = {}
    for i, class_name in enumerate(test_loader.dataset.classes):
        class_f1 = f1_score(y_true, y_pred, average='macro', labels=[i])
        class_f1_scores[class_name] = class_f1

    return accuracy, f1, precision, recall, class_f1_scores

In [ ]:
# Create a combined result DataFrame for all the datasets
combined_result_df = pd.DataFrame(columns=['Dataset', 'Accuracy', 'F1 Score', 'Precision', 'Recall'])

class_f1_scores = {}

for dataset_dir in dataset_dirs:
    test_dataset = ImageFolder(root=dataset_dir, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, pin_memory=True)

    path_components = os.path.normpath(dataset_dir).split(os.path.sep)
    dataset_name = os.path.join(*path_components[-3:])
    dataset_name = dataset_name.replace(os.path.sep, '_')

    accuracy, f1, precision, recall, individual_class_f1_scores = evaluate_model(model, test_loader, dataset_name)


    new_row = {'Dataset': dataset_name,
            'Accuracy': accuracy,
            'F1 Score': f1,
            'Precision': precision,
            'Recall': recall}
    combined_result_df = pd.concat([combined_result_df, pd.DataFrame([new_row])], ignore_index=True)
    print(combined_result_df)

    # Append the class F1 scores to the dictionary
    class_f1_scores[dataset_name] = individual_class_f1_scores

In [ ]:
# Save the combined result DataFrame to a text file
# Append individual F1 scores to the text file
with open(combined_result_file_path, 'a') as file:
    for dataset_name, f1_scores_dict in class_f1_scores.items():
        file.write("\nCombined Results:\n")
        file.write("--------------------------------------\n")
        file.write("Dataset\t\t\t\t\t\tAccuracy\t\tF1 Score\tPrecision\t\tRecall\n")
        file.write("--------------------------------------\n")

        dataset_results = combined_result_df[combined_result_df['Dataset'] == dataset_name]
        for index, row in dataset_results.iterrows():
            file.write(f"{row['Dataset']}\t{row['Accuracy']}\t{row['F1 Score']}\t{row['Precision']}\t{row['Recall']}\n")

        file.write("\nIndividual F1 Scores:\n")
        file.write("--------------------------------------\n")

        for class_name, class_f1 in f1_scores_dict.items():
            file.write(f"{class_name}: {class_f1}\n")

        file.write("\n--------------------------------------\n")

# Print message after processing all the datasets
print('Evaluation complete. Combined results and individual F1 scores saved in:', results_path)
